# 说明

在开发环境中将目录中的 `model目录/V版本`，`notebook目录` 进行 Sync Obs 同步，之后在下面代码里修改同步模型的版本号。


- `model目录/V版本` 中可能含有多个版本，只需要选择需要调试的版本目录进行同步即可

- `notebook目录` 中含 `detect_image.py文件` 和 `detect_video.py文件` 是便于自己本机运行和在开发环境 `Terminal - TensorFlow-1.13.1` 中执行。

在 `Terminal` 中使用命令 `source /home/ma-user/anaconda3/bin/activate TensorFlow-1.13.1` 可以切换到 `TensorFlow-1.13.1` 的环境方便运行


## 查看同步的模型目录和调试所需文件

In [ ]:
!ls -lh model/V0025/model

!ls -lh notebook

## 图片识别 `detect_image.py` 文件

In [ ]:
!python notebook/detect_image.py --image notebook/test.jpg --min_score 0.2 --show_image true --show_box_label true --input_size 1024 --version V0025 --h5 train_mask_rcnn.h5

## 视频识别 `detect_video.py` 文件

In [ ]:
!python notebook/detect_video.py --video notebook/test.mp4 --min_score 0.2 --show_image true --show_box_label true --input_size 1024 --version V0025 --h5 train_mask_rcnn.h5

## 下面是便于观察的ipynb调试

----

### 开始

detect_image.py文件 和 detect_video.py文件，所需模型依赖。

**参数配置**

- version 模型对应版本
- h5 模型文件名
- input_size 统一输入图像大小


In [ ]:
import os, sys
import tensorflow as tf
from moxing.framework import file
import cv2.cv2
import time

%matplotlib inline
import matplotlib.pyplot as plt 

# 参数配置
version = 'V0025' # 模型版本 
h5 = 'train_mask_rcnn.h5' # 模型文件
input_size = 1024 # 统一输入图像大小

# 执行所在路径， V0xxx 表示模型版本号
source_path = os.path.join(os.getcwd(), "model/" + version + "/model")
sys.path.append(source_path)

from draw import model_load, display_instances, sliceImage, image_to_base64

# 选择输出绘制类识别80类，
COCO_CLASSES = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
                'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign',
                'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
                'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag',
                'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite',
                'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
                'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
                'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair',
                'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
                'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock',
                'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

# 模型权重文件
COCO_MODEL_PATH = os.path.join(source_path, h5)

# 加载模型
model = model_load(COCO_MODEL_PATH, input_size)


### 图片识别

detect_image.py 文件内部代码

**参数配置**

- image 用于识别图文件
- min_score 显示最低分数 
- show_image 显示图片
- show_box_label 显示标识边框标签


In [ ]:
# 参数配置
image = cv2.imread('notebook/test.jpg') # 用于识别图文件
image_orig = image.copy() # 用于分割小图的原图
min_score = 0.2 # 显示最低分数
show_image = True # 显示图片beas64
show_box_label = True # 显示标识边框标签 

created_at = str(round(time.time() * 1000))

# obs桶路径
obs_path = "obs://puddings/ma-mask-rcnn/notebook/out/image/" + created_at

# 输出目录
out_path = "notebook/out/image/" + created_at

# 输出目录存在需要删除里边的内容
if os.path.exists(out_path):
    file.remove(out_path, recursive=True)
os.makedirs(out_path)

prev_time = time.time()
# 模型识别结果 rois, masks, class_ids, scores
results = model.detect([image], verbose=0)[0]
    
# 结果绘制到图
image, recognizer = display_instances(
    image,
    results,
    COCO_CLASSES,
    min_score,
    show_image,
    show_box_label
)

# 绘制时间
curr_time = time.time()
exec_time = curr_time - prev_time
print("识别耗时: %.2f ms" %(1000*exec_time))

# print("识别结果：", recognizer)

# 输入图片np uint8 尺寸/2
# x, y = image.shape[0:2]
# image = cv2.resize(image, (int(y / 2), int(x / 2)))

# base图片编码
# itb64 = image_to_base64(image)
# print(itb64)

# 绘制识别统计
totalStr = ""
for k in recognizer.keys():
    totalStr += '%s: %d    ' % (k, len(recognizer[k]))
cv2.putText(image, totalStr, (10,20), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (50, 0, 255), 1, cv2.LINE_AA)
    
# 绘制保存
cv2.imwrite(out_path + "/output_result.jpg", image)
cv2.imwrite(out_path + "/output_orig.jpg", image_orig)

# 切割识别到的物体
sliceImage(image_orig, recognizer, out_path)
   
# 复制保存到桶
print("输出目录：" + out_path)
file.copy_parallel(out_path, obs_path)

# 总图绘制显示ipynb
result = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
plt.figure(figsize=(10,10))
plt.imshow(result)
plt.axis('on')
plt.show()


### 视频识别

detect_video.py 文件内部代码

**参数配置**

- video 视频文件 
- min_score 显示最低分数 
- show_image 显示图片
- show_box_label 显示标识边框标签

In [ ]:
# 参数配置
video = cv2.VideoCapture('notebook/test.mp4') # 用于识别的视频文件 
min_score = 0.2 # 显示最低分数
show_image = True # 显示图片beas64
show_box_label = True # 显示标识边框标签 

# 输出目录
out_path = "notebook/out/video"

# 输出目录存在需要删除里边的内容
if os.path.exists(out_path):
    file.remove(out_path, recursive=True)
os.makedirs(out_path)

# 帧数，用于通过帧数取图
frameNum = 0

# 视频总帧统计物体数，存在重复
totalCount = {}

# obs桶路径
obs_path = "obs://puddings/ma-mask-rcnn/notebook/out/video"

# 保存统计总数并复制保存到桶
def outTotalObs(totalCount, out_path, obs_path):
    # 打开文件进行视频识别物总统计
    totalFile = open(out_path + "/totalCount.txt","w")
    # 文件统计写入
    for k in totalCount.keys():
        labelStr = "{0}：{1} \n".format(k, totalCount[k])
        totalFile.write(labelStr)
     # 关闭文件统计        
    totalFile.close()
    # 复制保存到桶
    file.copy_parallel(out_path, obs_path)

# 输出保存视频
fourcc = cv2.VideoWriter_fourcc(*'XVID')
fps = video.get(cv2.CAP_PROP_FPS)
size = (int(video.get(cv2.CAP_PROP_FRAME_WIDTH)), int(video.get(cv2.CAP_PROP_FRAME_HEIGHT)))
video_out = cv2.VideoWriter(out_path + "/outputVideo.mp4", fourcc, fps, size)

# 视频是否可以打开，进行逐帧识别绘制
while video.isOpened:
    # 视频读取图片帧
    retval, frame = video.read()
    if retval:
        frame_orig = frame.copy()
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    else:
        # 保存统计总数并复制保存到桶
        outTotalObs(totalCount, out_path, obs_path)
        # 读取失败、结束后释放所有内容
        video.release()
        video_out.release()
        print("没有图像！尝试使用其他视频")
        break

    print('识别帧：%d/%d' % (frameNum, video.get(7)))
    prev_time = time.time()
    # 模型识别结果 rois, masks, class_ids, scores
    results = model.detect([frame], verbose=0)[0]

    # 结果绘制到图
    image, recognizer = display_instances(
        frame,
        results,
        COCO_CLASSES,
        min_score,
        show_image,
        show_box_label
    )

    # 绘制时间
    curr_time = time.time()
    exec_time = curr_time - prev_time
    print("识别耗时: %.2f ms" %(1000*exec_time))
        
    # print("识别结果：", recognizer)
    
    # 遍历识别数据并绘制帧识别统计
    totalStr = ""
    for k in recognizer.keys():
        if k not in totalCount: totalCount[k] = 0
        num = len(recognizer[k]);
        totalCount[k] += num
        totalStr += '%s: %d    ' % (k, num)
        cv2.putText(image, totalStr, (10,20), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (50, 0, 255), 1, cv2.LINE_AA)

    # 视频输出保存
    result = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    video_out.write(result)
    
    # 每300帧取图进行分割保存
    if(frameNum % 300 == 0):
        # 输出帧目录,不存目录需要创建
        slice_path = os.path.join(out_path, "imageSeg-" + str(frameNum))
        if not os.path.exists(slice_path):
            os.makedirs(slice_path)
        # 绘制帧保存
        cv2.imwrite(os.path.join(slice_path, "output_result.jpg"), result)
        cv2.imwrite(os.path.join(slice_path, "output_orig.jpg"), frame_orig)
        # 切割识别到的物体
        sliceImage(frame_orig, recognizer, slice_path)
    frameNum += 1

    # 保存统计总数并复制保存到桶
    outTotalObs(totalCount, out_path, obs_path)

    # 绘制结果ipynb显示
    plt.figure(figsize=(10,10))
    plt.imshow(image)
    plt.axis('on')
    plt.show()

# 保存统计总数并复制保存到桶
print("输出目录：" + out_path)
outTotalObs(totalCount, out_path, obs_path)

# 任务完成后释放所有内容
video.release()
video_out.release()
